In [1]:
!nvidia-smi

Mon Jul  8 21:08:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0              27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -U transformers accelerate bitsandbytes

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-08 21:08:38--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-08 21:08:38 (52.3 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [4]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
!pip install huggingface_hub

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from transformers import AutoTokenizer, Gemma2ForCausalLM

## Q2. Downloading an LLM

In [11]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_auth_token=True)
model = Gemma2ForCausalLM.from_pretrained("google/gemma-2b", device_map="cuda:0", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3101: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You are using a mod

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Gemma2ForCausalLM were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['model.layers.0.post_feedforward_layernorm.weight', 'model.layers.0.pre_feedforward_layernorm.weight', 'model.layers.1.post_feedforward_layernorm.weight', 'model.layers.1.pre_feedforward_layernorm.weight', 'model.layers.10.post_feedforward_layernorm.weight', 'model.layers.10.pre_feedforward_layernorm.weight', 'model.layers.11.post_feedforward_layernorm.weight', 'model.layers.11.pre_feedforward_layernorm.weight', 'model.layers.12.post_feedforward_layernorm.weight', 'model.layers.12.pre_feedforward_layernorm.weight', 'model.layers.13.post_feedforward_layernorm.weight', 'model.layers.13.pre_feedforward_layernorm.weight', 'model.layers.14.post_feedforward_layernorm.weight', 'model.layers.14.pre_feedforward_layernorm.weight', 'model.layers.15.post_feedforward_layernorm.weight', 'model.layers.15.pre_feedforward_layernorm.weight', 'model.layers.16.post_feedforward_la

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [12]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

## Q3. Running the LLM

In [13]:
llm('10 * 10')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'10 * 10<unused0><unused0><unused0><unused0>'

## Q4. Donwloading the weights

In [14]:
!ls -la ~/.cache/huggingface/hub

total 24
drwxr-xr-x 5 root root 4096 Jul  8 21:07 .
drwxr-xr-x 3 root root 4096 Jul  8 20:29 ..
drwxr-xr-x 4 root root 4096 Jul  8 21:07 .locks
drwxr-xr-x 6 root root 4096 Jul  8 20:39 models--google--gemma-2-9b
drwxr-xr-x 6 root root 4096 Jul  8 21:09 models--google--gemma-2b
-rw-r--r-- 1 root root    1 Jul  8 20:19 version.txt


In [15]:
!du -h ~/.cache/huggingface/hub

35G	/root/.cache/huggingface/hub/models--google--gemma-2-9b/blobs
8.0K	/root/.cache/huggingface/hub/models--google--gemma-2-9b/refs
4.0K	/root/.cache/huggingface/hub/models--google--gemma-2-9b/.no_exist/7725a3ee78e8d86c51efeb1f801a195dfa91c715
8.0K	/root/.cache/huggingface/hub/models--google--gemma-2-9b/.no_exist
44K	/root/.cache/huggingface/hub/models--google--gemma-2-9b/snapshots/7725a3ee78e8d86c51efeb1f801a195dfa91c715
48K	/root/.cache/huggingface/hub/models--google--gemma-2-9b/snapshots
35G	/root/.cache/huggingface/hub/models--google--gemma-2-9b
4.7G	/root/.cache/huggingface/hub/models--google--gemma-2b/blobs
8.0K	/root/.cache/huggingface/hub/models--google--gemma-2b/refs
4.0K	/root/.cache/huggingface/hub/models--google--gemma-2b/.no_exist/68e273d91b1d6ea57c9e6024c4f887832f7b43fa
8.0K	/root/.cache/huggingface/hub/models--google--gemma-2b/.no_exist
20K	/root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa
24K	/root/.cache/huggingfac

## Q6. Serving it

In [17]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 0.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [18]:
response = llm("What's the formula for energy?")
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"What's the formula for energy?<unused0><unused0><unused0><unused0>"

In [19]:
len(response)

66